<a href="https://colab.research.google.com/github/jsansao/idl/blob/main/TEIC_Licao30_Similaridade_sentenca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Análise de Similaridade de Sentenças com Transformers

Este notebook demonstra como usar a biblioteca `sentence-transformers` (construída sobre o Hugging Face Transformers) para calcular a **similaridade semântica** entre sentenças em português.

A similaridade semântica vai além da simples correspondência de palavras-chave. Ela tenta entender o *significado* por trás das sentenças.

Usaremos um modelo pré-treinado que é bom em várias línguas, incluindo o português, e que foi otimizado para tarefas de similaridade.

In [1]:
# Instala a biblioteca sentence-transformers
# O -q (quiet) é para reduzir a quantidade de logs de instalação
!pip install -q sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer, util
import torch

# Carrega um modelo pré-treinado.
# 'paraphrase-multilingual-MiniLM-L12-v2' é um modelo leve, rápido e
# excelente para tarefas de similaridade semântica em mais de 50 idiomas.
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

print("Modelo carregado com sucesso!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo carregado com sucesso!


## Passo 1: Gerar Embeddings (Representações Vetoriais)

Para que o modelo entenda as sentenças, ele as converte em vetores numéricos de tamanho fixo, chamados "embeddings". Sentenças com significados semelhantes terão vetores "próximos" no espaço vetorial.

Vamos definir nossa lista de sentenças.

In [3]:
# Lista de sentenças que queremos comparar
sentences = [
    "O cachorro correu no parque.",             # Sentença 0
    "Um cão estava correndo no gramado.",       # Sentença 1 (Similar a 0)
    "O gato está dormindo no sofá.",            # Sentença 2 (Diferente)
    "Aquele menino joga futebol muito bem.",    # Sentença 3 (Diferente)
    "Qual é a capital da França?",              # Sentença 4 (Diferente)
    "Paris é a capital francesa."               # Sentença 5 (Relacionada a 4)
]

# Gera os embeddings para todas as sentenças
# O modelo.encode() transforma o texto em vetores
embeddings = model.encode(sentences, convert_to_tensor=True)

# Vamos verificar a forma (shape) dos nossos embeddings
# Deve ser [número_de_sentenças, dimensão_do_modelo]
# Para este modelo, a dimensão é 384
print("Shape dos Embeddings:", embeddings.shape)

Shape dos Embeddings: torch.Size([6, 384])


## Passo 2: Calcular a Similaridade de Cosseno

Agora que temos os vetores, podemos calcular a "distância" entre eles. A métrica mais comum para isso é a **Similaridade de Cosseno (Cosine Similarity)**.

* Um score de **1.0** significa que as sentenças são idênticas (ou semanticamente muito parecidas).
* Um score de **0.0** significa que não há relação.
* Um score de **-1.0** significa que têm significados opostos.

(Com este tipo de modelo, os scores geralmente ficam entre 0 e 1).

Vamos calcular uma matriz de similaridade, comparando cada sentença com todas as outras.

In [6]:
# Calcula a similaridade de cosseno entre todos os pares de embeddings
# A função util.cos_sim faz isso de forma eficiente
cosine_scores = util.cos_sim(embeddings, embeddings)

# Exibe a matriz de similaridade
# A linha 'i' e coluna 'j' mostra a similaridade entre a sentença 'i' e a 'j'
print("Matriz de Similaridade:")
print(cosine_scores)

print("\n--- Análise Detalhada (Score > 0.6) ---")
# Vamos iterar pela matriz para ver os pares mais similares
for i in range(len(sentences)):
    for j in range(i + 1, len(sentences)): # (i+1) para evitar duplicatas e auto-comparações
        score = cosine_scores[i][j]
        if score > 0.6: # Mostra apenas pares com alta similaridade
            print(f"Sentença 1: {sentences[i]}")
            print(f"Sentença 2: {sentences[j]}")
            print(f"Similaridade: {score:.4f}\n")

Matriz de Similaridade:
tensor([[ 1.0000,  0.8005, -0.0558, -0.0167,  0.1219,  0.1383],
        [ 0.8005,  1.0000, -0.0248,  0.0259,  0.0938,  0.0912],
        [-0.0558, -0.0248,  1.0000,  0.0464, -0.0430, -0.0327],
        [-0.0167,  0.0259,  0.0464,  1.0000,  0.0921,  0.0451],
        [ 0.1219,  0.0938, -0.0430,  0.0921,  1.0000,  0.8355],
        [ 0.1383,  0.0912, -0.0327,  0.0451,  0.8355,  1.0000]])

--- Análise Detalhada (Score > 0.6) ---
Sentença 1: O cachorro correu no parque.
Sentença 2: Um cão estava correndo no gramado.
Similaridade: 0.8005

Sentença 1: Qual é a capital da França?
Sentença 2: Paris é a capital francesa.
Similaridade: 0.8355



### Análise dos Resultados

Você deve ter notado scores altos nos seguintes pares:

1.  **"O cachorro correu no parque."** e **"Um cão estava correndo no gramado."**
    * *Score alto (provavelmente > 0.8)*. O modelo entende que "cachorro" e "cão" são sinônimos e que "parque" e "gramado" são locais relacionados.

2.  **"Qual é a capital da França?"** e **"Paris é a capital francesa."**
    * *Score alto (provavelmente > 0.7)*. Embora uma seja pergunta e a outra resposta, elas tratam exatamente do mesmo tópico semântico (Capital da França).

3.  Os outros pares, como "O gato está dormindo..." e "O cachorro correu...", terão scores baixos, pois tratam de assuntos diferentes.

## Passo 3: Exemplo Prático (Busca Semântica)

Um uso muito comum disso é encontrar a sentença mais relevante em um "banco de dados" (corpus) para uma nova sentença (query).

Imagine que as `sentences` da Célula 5 são documentos em nosso banco de dados. Agora, temos uma nova pergunta (query) e queremos encontrar a resposta ou a sentença mais parecida.

In [5]:
# Nosso "banco de dados" (corpus) é a mesma lista de antes
corpus = [
    "O cachorro correu no parque.",
    "Um cão estava correndo no gramado.",
    "O gato está dormindo no sofá.",
    "Aquele menino joga futebol muito bem.",
    "Qual é a capital da França?",
    "Paris é a capital francesa."
]

# A sentença de "busca" (query)
query = "Onde fica a capital da França?"

# 1. Gera o embedding da query
query_embedding = model.encode(query, convert_to_tensor=True)

# 2. Gera os embeddings do corpus (já fizemos isso, mas vamos fazer de novo para
#    manter o exemplo contido. Em uma aplicação real, você armazenaria os embeddings do corpus)
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# 3. Calcula a similaridade da query (1 vetor) contra o corpus (N vetores)
#    Isso nos dará N scores.
semantic_scores = util.cos_sim(query_embedding, corpus_embeddings)[0] # Pegamos [0] pois a query é única

# 4. Combina os scores com as sentenças
scored_sentences = sorted(zip(semantic_scores, corpus), reverse=True) # Ordena do mais similar para o menos

# 5. Mostra os resultados
print(f"Query: {query}\n")
print("Resultados da busca semântica (top 3):")

for score, sentence in scored_sentences[:3]:
    # A biblioteca 'torch' nos dá tensores, usamos .item() para pegar o número Python puro
    print(f"  Score: {score.item():.4f}  |  Sentença: {sentence}")

Query: Onde fica a capital da França?

Resultados da busca semântica (top 3):
  Score: 0.9900  |  Sentença: Qual é a capital da França?
  Score: 0.8307  |  Sentença: Paris é a capital francesa.
  Score: 0.1288  |  Sentença: O cachorro correu no parque.


### Análise da Busca

Mesmo que a query "Onde fica a capital da França?" não use as palavras "Paris" ou "francesa", o modelo entende o significado e corretamente identifica "Qual é a capital da França?" e "Paris é a capital francesa." como as sentenças mais relevantes semanticamente.

Isso demonstra o poder dos Transformers para entender o *significado* além das palavras-chave.